#### Categorical Variables

One of the main ways for working with categorical variables is using 0, 1 encodings.  In this technique, you create a new column for every level of the categorical variable.  The **advantages** of this approach include:

1. The ability to have differing influences of each level on the response.
2. You do not impose a rank of the categories.
3. The ability to interpret the results more easily than other encodings.

The **disadvantages** of this approach are that you introduce a large number of effects into your model.  If you have a large number of categorical variables or categorical variables with a large number of levels, but not a large sample size, you might not be able to estimate the impact of each of these variables on your response variable.  There are some rules of thumb that suggest 10 data points for each variable you add to your model.  That is 10 rows for each column.  This is a reasonable lower bound, but the larger your sample (assuming it is representative), the better.

Let's try out adding dummy variables for the categorical variables into the model.  We will compare to see the improvement over the original model only using quantitative variables.  


#### Run the two cells below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import test3 as t
import seaborn as sns
%matplotlib inline

df = pd.read_csv('./survey_results_public.csv')
df.head()

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#Only use quant variables and drop any rows with missing values
num_vars = df[['Salary', 'CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]

#Drop the rows with missing salaries
drop_sal_df = num_vars.dropna(subset=['Salary'], axis=0)

# Mean function
fill_mean = lambda col: col.fillna(col.mean())
# Fill the mean
fill_df = drop_sal_df.apply(fill_mean, axis=0)

#Split into explanatory and response variables
X = fill_df[['CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]
y = fill_df['Salary']

#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42) 

lm_model = LinearRegression(normalize=True) # Instantiate
lm_model.fit(X_train, y_train) #Fit
        
#Predict and score the model
y_test_preds = lm_model.predict(X_test) 
"The r-squared score for the model using only quantitative variables was {} on {} values.".format(r2_score(y_test, y_test_preds), len(y_test))

'The r-squared score for the model using only quantitative variables was 0.03257139063404413 on 1503 values.'

#### Question 1

**1.** Use the **df** dataframe. Identify the columns that are categorical in nature.  How many of the columns are considered categorical?  Use the reference [here](http://pbpython.com/categorical-encoding.html) if you get stuck.

In [3]:
df.dtypes

Respondent                       int64
Professional                    object
ProgramHobby                    object
Country                         object
University                      object
EmploymentStatus                object
FormalEducation                 object
MajorUndergrad                  object
HomeRemote                      object
CompanySize                     object
CompanyType                     object
YearsProgram                    object
YearsCodedJob                   object
YearsCodedJobPast               object
DeveloperType                   object
WebDeveloperType                object
MobileDeveloperType             object
NonDeveloperType                object
CareerSatisfaction             float64
JobSatisfaction                float64
ExCoderReturn                   object
ExCoderNotForMe                 object
ExCoderBalance                  object
ExCoder10Years                  object
ExCoderBelonged                 object
ExCoderSkills            

In [4]:
cat_df = df.select_dtypes(include=['object']).copy() # Subset to a dataframe only holding the categorical columns

# Print how many categorical columns are in the dataframe - should be 147
cat_df.shape[1]

147

In [5]:
# Test your dataframe matches the solution
t.cat_df_check(cat_df)

Nice job! That looks right!


#### Question 2

**2.** Use **cat_df** and the cells below to fill in the dictionary below the correct value for each statement.

In [6]:
len(set(cat_df.columns[cat_df.isnull().mean()==0]))

6

In [7]:
len(cat_df.columns[cat_df.isnull().sum()/len(cat_df) > .50])

49

In [8]:
len(cat_df.columns[cat_df.isnull().sum()/len(cat_df) > .75])

13

In [9]:
# Provide the key as an `integer` that answers the question

cat_df_dict = {'the number of columns with no missing values': 6, #int here, 
               'the number of columns with more than half of the column missing': 50,#int here,
               'the number of columns with more than 75% of the column missing':13,  #int here
}

# Check your dictionary results
t.cat_df_dict_check(cat_df_dict)

Nice job! That looks right to me!


#### Question 3

**3.** For each of the categorical variables, we now need to create dummy columns.  However, as we saw above, there are a lot of missing values in the current set of categorical columns.  So, you might be wondering, what happens when you dummy a column that has missing values.

The documentation for creating dummy variables in pandas is available [here](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.get_dummies.html), but we can also just put this to practice to see what happens.

First, run the cell below to create a dataset that you will use before moving to the full stackoverflow data.

After you have created **dummy_var_df**, use the additional cells to fill in the **sol_3_dict** with the correct variables that match each key.

In [10]:
dummy_var_df = pd.DataFrame({'col1': ['a', 'a', 'b', 'b', 'a', np.nan, 'b', np.nan],
                             'col2': [1, np.nan, 3, np.nan, 5, 6, 7, 8] 
})
                            
dummy_var_df

,col1,col2
0,a,1.0
1,a,NaN
2,b,3.0
3,b,NaN
4,a,5.0
5,NaN,6.0
6,b,7.0
7,NaN,8.0


In [11]:
pd.get_dummies(dummy_var_df['col1'])# Use this cell to write whatever code you need.

,a,b
0,1,0
1,1,0
2,0,1
3,0,1
4,1,0
5,0,0
6,0,1
7,0,0


In [12]:
a = 1
b = 2
c = 3
d = 'col1'
e = 'col2'
f = 'the rows with NaNs are dropped by default'
g = 'the NaNs are always encoded as 0'


sol_3_dict = {'Which column should you create a dummy variable for?': d,#Letter here,
              'When you use the default settings for creating dummy variables, how many are created?':b, #Letter here,
              'What happens with the nan values?':g #Letter here 
             }

# Check your dictionary against the solution
t.sol_3_dict_check(sol_3_dict)

Nice job! That looks right to me!


#### Question 4

**4.** Notice, you can also use **get_dummies** to encode **NaN** values as their own dummy coded column using the **dummy_na** argument.  Often these NaN values are also informative, but you are not capturing them by leaving them as 0 in every column of your model.

Create a new encoding for **col1** of **dummy_var_df** that provides dummy columns not only for each level, but also for the missing values below. Store the 3 resulting dummy columns in **dummy_cols_df** and check your solution against ours.

In [13]:
dummy_cols_df = pd.get_dummies(dummy_var_df['col1'],dummy_na=True)#Create the three dummy columns for dummy_var_df

# Look at your result
dummy_cols_df

,a,b,nan
0,1,0,0
1,1,0,0
2,0,1,0
3,0,1,0
4,1,0,0
5,0,0,1
6,0,1,0
7,0,0,1


In [14]:
# Check against the solution
t.dummy_cols_df_check(dummy_cols_df)

Nice job! That looks right!


In [15]:
pd.get_dummies(dummy_var_df, ['col1'],dummy_na=True)

,col2,col1_a,col1_b,col1_nan
0,1.0,1,0,0
1,NaN,1,0,0
2,3.0,0,1,0
3,NaN,0,1,0
4,5.0,1,0,0
5,6.0,0,0,1
6,7.0,0,1,0
7,8.0,0,0,1


#### Question 5

**5.** We could use either of the above to begin creating an X matrix that would (potentially) allow us to predict better than just the numeric columns we have been using thus far.

First, complete the **create_dummy_df**.  Follow the instructions in the document string to assist as necessary.

In [16]:
'''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''

'\n    INPUT:\n    df - pandas dataframe with categorical variables you want to dummy\n    cat_cols - list of strings that are associated with names of the categorical columns\n    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not\n    \n    OUTPUT:\n    df - a new dataframe that has the following characteristics:\n            1. contains all columns that were not specified as categorical\n            2. removes all the original columns in cat_cols\n            3. dummy columns for each of the categorical columns in cat_cols\n            4. if dummy_na is True - it also contains dummy columns for the NaN values\n            5. Use a prefix of the column name with an underscore (_) for separating \n    '

In [17]:
#Create a copy of the dataframe
cat_df_copy = cat_df.copy()
#Pull a list of the column names of the categorical variables
cat_cols_lst = cat_df.columns

def create_dummy_df(df, cat_cols, dummy_na):
        cat_cols = cat_cols.tolist()
        if dummy_na == True:
            df = pd.get_dummies(df, columns = cat_cols,dummy_na=True,drop_first=True)
        else:
            df = pd.get_dummies(df, columns = cat_cols,drop_first=True)
        return df


In [18]:
cat_cols_lst.tolist()

['Professional',
 'ProgramHobby',
 'Country',
 'University',
 'EmploymentStatus',
 'FormalEducation',
 'MajorUndergrad',
 'HomeRemote',
 'CompanySize',
 'CompanyType',
 'YearsProgram',
 'YearsCodedJob',
 'YearsCodedJobPast',
 'DeveloperType',
 'WebDeveloperType',
 'MobileDeveloperType',
 'NonDeveloperType',
 'ExCoderReturn',
 'ExCoderNotForMe',
 'ExCoderBalance',
 'ExCoder10Years',
 'ExCoderBelonged',
 'ExCoderSkills',
 'ExCoderWillNotCode',
 'ExCoderActive',
 'PronounceGIF',
 'ProblemSolving',
 'BuildingThings',
 'LearningNewTech',
 'BoringDetails',
 'JobSecurity',
 'DiversityImportant',
 'AnnoyingUI',
 'FriendsDevelopers',
 'RightWrongWay',
 'UnderstandComputers',
 'SeriousWork',
 'InvestTimeTools',
 'WorkPayCare',
 'KinshipDevelopers',
 'ChallengeMyself',
 'CompetePeers',
 'ChangeWorld',
 'JobSeekingStatus',
 'LastNewJob',
 'AssessJobIndustry',
 'AssessJobRole',
 'AssessJobExp',
 'AssessJobDept',
 'AssessJobTech',
 'AssessJobProjects',
 'AssessJobCompensation',
 'AssessJobOffice',
 

In [19]:
#Dropping where the salary has missing values
df  = df.dropna(subset=['Salary'], axis=0)
#Create a copy of the dataframe
cat_df_copy = cat_df.copy()
#Pull a list of the column names of the categorical variables
cat_cols_lst = cat_df.columns

def create_dummy_df(df, cat_cols, dummy_na):
        cat_cols = cat_cols.tolist()
        if dummy_na == True:
            df = pd.get_dummies(df, columns = cat_cols,dummy_na=True,drop_first=True)
        else:
            df = pd.get_dummies(df, columns = cat_cols,drop_first=True)
        return df
    
df_new = create_dummy_df(df, cat_cols_lst, dummy_na=False) #Use your newly created function

# Show shape to assure it has a shape of (5009, 11938)
print(df_new.shape)

(5009, 11938)


In [20]:
def create_dummy_df2(df, cat_cols, dummy_na):
    for col in  cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, 
                        prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df

df_new2 = create_dummy_df2(df, cat_cols_lst, dummy_na=False) #Use your newly created function
print(df_new2.shape)

'''def create_dummy_df(df, cat_cols, dummy_na):
        cat_cols = cat_cols.tolist()
        if dummy_na == True:
            df = pd.get_dummies(df, columns = cat_cols,dummy_na=True,drop_first=True)
        else:
            df = pd.get_dummies(df, columns = cat_cols,drop_first=True)
        return df'''

(5009, 11938)


In [21]:
[item for item in df_new.columns if item not in df_new2.columns]
#checking if there're differences btw two approaches

[]

In [22]:
%%time
cat_cols_lst

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.87 µs


Index(['Professional', 'ProgramHobby', 'Country', 'University',
       'EmploymentStatus', 'FormalEducation', 'MajorUndergrad', 'HomeRemote',
       'CompanySize', 'CompanyType',
       ...
       'StackOverflowBetter', 'StackOverflowWhatDo', 'StackOverflowMakeMoney',
       'Gender', 'HighestEducationParents', 'Race', 'SurveyLong',
       'QuestionsInteresting', 'QuestionsConfusing', 'InterestedAnswers'],
      dtype='object', length=147)

In [23]:
df_new.head()

,Respondent,CareerSatisfaction,JobSatisfaction,HoursPerWeek,StackOverflowSatisfaction,Salary,ExpectedSalary,"ProgramHobby_Yes, I contribute to open source projects","ProgramHobby_Yes, I program as a hobby","ProgramHobby_Yes, both",...,QuestionsInteresting_Strongly agree,QuestionsInteresting_Strongly disagree,QuestionsConfusing_Disagree,QuestionsConfusing_Somewhat agree,QuestionsConfusing_Strongly agree,QuestionsConfusing_Strongly disagree,InterestedAnswers_Disagree,InterestedAnswers_Somewhat agree,InterestedAnswers_Strongly agree,InterestedAnswers_Strongly disagree
2,3,8.0,9.0,NaN,8.0,113750.0,NaN,0,0,1,...,0,0,1,0,0,0,0,0,0,0
14,15,8.0,8.0,NaN,8.0,100000.0,NaN,0,1,0,...,0,0,1,0,0,0,0,0,0,0
17,18,9.0,8.0,NaN,8.0,130000.0,NaN,0,0,1,...,0,0,1,0,0,0,0,0,0,0
18,19,5.0,3.0,NaN,NaN,82500.0,NaN,0,1,0,...,0,0,0,0,0,0,0,0,0,0
22,23,8.0,9.0,NaN,8.0,100764.0,NaN,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [24]:
df_new.columns.tolist()

['Respondent',
 'CareerSatisfaction',
 'JobSatisfaction',
 'HoursPerWeek',
 'StackOverflowSatisfaction',
 'Salary',
 'ExpectedSalary',
 'ProgramHobby_Yes, I contribute to open source projects',
 'ProgramHobby_Yes, I program as a hobby',
 'ProgramHobby_Yes, both',
 'Country_Albania',
 'Country_Argentina',
 'Country_Armenia',
 'Country_Australia',
 'Country_Austria',
 'Country_Azerbaidjan',
 'Country_Bangladesh',
 'Country_Belarus',
 'Country_Belgium',
 'Country_Bermuda',
 'Country_Bosnia-Herzegovina',
 'Country_Brazil',
 'Country_Bulgaria',
 'Country_Canada',
 'Country_Chile',
 'Country_China',
 'Country_Colombia',
 'Country_Costa Rica',
 'Country_Croatia',
 'Country_Cyprus',
 'Country_Czech Republic',
 'Country_Denmark',
 'Country_Dominican Republic',
 'Country_Ecuador',
 'Country_Egypt',
 'Country_El Salvador',
 'Country_Estonia',
 'Country_Finland',
 'Country_France',
 'Country_Georgia',
 'Country_Germany',
 'Country_Ghana',
 'Country_Greece',
 'Country_Honduras',
 'Country_Hong Kong

#### Question 6

**6.** Use the document string below to complete the function.  Then test your function against the solution.  

In [26]:
   '''
    INPUT:
    df - a dataframe holding all the variables of interest
    response_col - a string holding the name of the column 
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    
    Your function should:
    1. Drop the rows with missing response values
    2. Drop columns with NaN for all the values
    3. Use create_dummy_df to dummy categorical columns
    4. Fill the mean of the column for any missing values 
    5. Split your data into an X matrix and a response vector y
    6. Create training and test sets of data
    7. Instantiate a LinearRegression model with normalized data
    8. Fit your model to the training data
    9. Predict the response for the training data and the test data
    10. Obtain an rsquared value for both the training and test data
    '''

'\n INPUT:\n df - a dataframe holding all the variables of interest\n response_col - a string holding the name of the column \n cat_cols - list of strings that are associated with names of the categorical columns\n dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not\n test_size - a float between [0,1] about what proportion of data should be in the test dataset\n rand_state - an int that is provided as the random state for splitting the data into training and test \n \n OUTPUT:\n test_score - float - r2 score on the test data\n train_score - float - r2 score on the test data\n lm_model - model object from sklearn\n X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model\n \n Your function should:\n 1. Drop the rows with missing response values\n 2. Drop columns with NaN for all the values\n 3. Use create_dummy_df to dummy categorical columns\n 4. Fill the mean of the column for any missing values \n 5. Split your

In [35]:
def clean_fit_linear_mod(df, response_col, cat_cols, dummy_na, test_size=.3, rand_state=42):
    
    df = df.dropna(subset=[response_col], axis=0)
    df.dropna(how='all', axis=1, inplace=True)
    df = create_dummy_df2(df, cat_cols, dummy_na)
    
    fill_mean = lambda x: x.fillna(x.mean())
    df = df.apply(fill_mean, axis=0)
    
    X = df.drop(response_col, axis=1)
    y = df[response_col]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=rand_state)
   
    lm_model = LinearRegression(normalize=True) 
    lm_model.fit(X_train, y_train) 
    
    #Predict using your model
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)

    #Score using your model
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)
    
    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test


#Test your function with the above dataset
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = clean_fit_linear_mod(df_new, 'Salary', cat_cols_lst, dummy_na=False)

#Print training and testing score
print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))

The rsquared on the training data was 1.0.  The rsquared on the test data was 0.4538301268288336.


Notice how much higher the rsquared value is on the training data than it is on the test data - why do you think that is?

In [25]:
v1 = df.drop('Salary', axis=1)
v1.columns

Index(['Respondent', 'Professional', 'ProgramHobby', 'Country', 'University',
       'EmploymentStatus', 'FormalEducation', 'MajorUndergrad', 'HomeRemote',
       'CompanySize',
       ...
       'StackOverflowWhatDo', 'StackOverflowMakeMoney', 'Gender',
       'HighestEducationParents', 'Race', 'SurveyLong', 'QuestionsInteresting',
       'QuestionsConfusing', 'InterestedAnswers', 'ExpectedSalary'],
      dtype='object', length=153)

In [26]:
v2 = df.loc[:, df.columns != 'Salary']
v2.columns

Index(['Respondent', 'Professional', 'ProgramHobby', 'Country', 'University',
       'EmploymentStatus', 'FormalEducation', 'MajorUndergrad', 'HomeRemote',
       'CompanySize',
       ...
       'StackOverflowWhatDo', 'StackOverflowMakeMoney', 'Gender',
       'HighestEducationParents', 'Race', 'SurveyLong', 'QuestionsInteresting',
       'QuestionsConfusing', 'InterestedAnswers', 'ExpectedSalary'],
      dtype='object', length=153)

In [28]:
[item for item in v1.columns if item not in v2.columns]

[]

In [39]:
%%time

def create_dummy_df2(df, cat_cols, dummy_na):
    for col in  cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, 
                        prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df

'''using the above since this function would break the code: def create_dummy_df(df, cat_cols, dummy_na):
        cat_cols = cat_cols.tolist()
        if dummy_na == True:
            df = pd.get_dummies(df, columns = cat_cols,dummy_na=True,drop_first=True)
        else:
            df = pd.get_dummies(df, columns = cat_cols,drop_first=True)
        return df'''


def clean_fit_linear_mod(df, response_col, cat_cols, dummy_na, test_size=.3, rand_state=42):
    
    df = df.dropna(subset=[response_col], axis=0)
    df.dropna(how='all', axis=1, inplace=True)
    df = create_dummy_df2(df, cat_cols, dummy_na)
    
    fill_mean = lambda x: x.fillna(x.mean())
    df = df.apply(fill_mean, axis=0)
    
    X = df.loc[:, df.columns != 'Salary'] #or df.drop('Salary', axis=1)
    y = df[response_col]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=rand_state)
   
    lm_model = LinearRegression(normalize=True) 
    lm_model.fit(X_train, y_train) 
    
    #Predict using your model
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)

    #Score using your model
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)
    
    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test

#Test your function with the above dataset
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = clean_fit_linear_mod(df_new, 'Salary', cat_cols_lst, dummy_na=False)

#Print training and testing score
print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))


The rsquared on the training data was 1.0.  The rsquared on the test data was 0.4538301268288336.
CPU times: user 3min 6s, sys: 2.38 s, total: 3min 8s
Wall time: 3min 9s


In [42]:
print(test_score, train_score)

0.453830126829 1.0


In [40]:
%%time
clean_fit_linear_mod(df_new, 'Salary', cat_cols_lst, dummy_na=False)

CPU times: user 3min 6s, sys: 2.37 s, total: 3min 8s
Wall time: 3min 9s


(0.45383012682883361,
 1.0,
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True),
        Respondent  CareerSatisfaction  JobSatisfaction  HoursPerWeek  \
 5717         5718           10.000000         6.000000      2.447415   
 4285         4286            7.000000         7.000000      2.447415   
 3093         3094            8.000000         7.000000      0.000000   
 11933       11934           10.000000        10.000000      2.447415   
 13949       13950            8.000000         8.000000      2.447415   
 17827       17828            8.000000         6.000000      2.447415   
 1671         1672           10.000000         7.000000      2.447415   
 13978       13979            5.000000         8.000000      1.000000   
 586           587            9.000000        10.000000      2.447415   
 14167       14168            3.000000         2.000000     10.000000   
 5295         5296            9.000000         8.000000      2.447415   
 10566       10567

In [41]:
def clean_fit_linear_mod(df, response_col, cat_cols, dummy_na, test_size=.3, rand_state=42):
    
    df = df.dropna(subset=[response_col], axis=0)
    df.dropna(how='all', axis=1, inplace=True)
    df = create_dummy_df(df, cat_cols, dummy_na)
    
    fill_mean = lambda x: x.fillna(x.mean())
    df = df.apply(fill_mean, axis=0)
    
    X = df.loc[:, df.columns != 'Salary'] #or df.drop('Salary', axis=1)
    y = df[response_col]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=rand_state)
   
    lm_model = LinearRegression(normalize=True) 
    lm_model.fit(X_train, y_train) 
    
    #Predict using your model
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)

    #Score using your model
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)
    
    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test

#Test your function with the above dataset
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = clean_fit_linear_mod(df_new, 'Salary', cat_cols_lst, dummy_na=False)

#Print training and testing score
print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))


KeyError: "['Professional' 'ProgramHobby' 'Country' 'University' 'EmploymentStatus'\n 'FormalEducation' 'MajorUndergrad' 'HomeRemote' 'CompanySize'\n 'CompanyType' 'YearsProgram' 'YearsCodedJob' 'YearsCodedJobPast'\n 'DeveloperType' 'WebDeveloperType' 'MobileDeveloperType'\n 'NonDeveloperType' 'ExCoderReturn' 'ExCoderNotForMe' 'ExCoderBalance'\n 'ExCoder10Years' 'ExCoderBelonged' 'ExCoderSkills' 'ExCoderWillNotCode'\n 'ExCoderActive' 'PronounceGIF' 'ProblemSolving' 'BuildingThings'\n 'LearningNewTech' 'BoringDetails' 'JobSecurity' 'DiversityImportant'\n 'AnnoyingUI' 'FriendsDevelopers' 'RightWrongWay' 'UnderstandComputers'\n 'SeriousWork' 'InvestTimeTools' 'WorkPayCare' 'KinshipDevelopers'\n 'ChallengeMyself' 'CompetePeers' 'ChangeWorld' 'JobSeekingStatus'\n 'LastNewJob' 'AssessJobIndustry' 'AssessJobRole' 'AssessJobExp'\n 'AssessJobDept' 'AssessJobTech' 'AssessJobProjects'\n 'AssessJobCompensation' 'AssessJobOffice' 'AssessJobCommute'\n 'AssessJobRemote' 'AssessJobLeaders' 'AssessJobP